In [3]:
import cv2
import numpy as np

def calculate_ink_density(image, contour):
    # Create a mask for the contour region
    mask = np.zeros_like(image, dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

    # Apply the mask to the image
    masked_image = cv2.bitwise_and(image, mask)

    # Convert the masked image to grayscale
    gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)

    # Calculate the ink density as the ratio of non-zero pixels to the total number of pixels in the contour region
    ink_density = np.count_nonzero(gray) / float(cv2.contourArea(contour))

    return ink_density

def flag_overwritten_text(image, threshold=0.5, min_contour_area=100):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to obtain a binary image
    threshold = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 4)

    # Find contours in the binary image
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank image for drawing the flagged contours
    flagged_image = np.copy(image)

    # Iterate through the contours
    for contour in contours:
        # Calculate the area of the contour
        contour_area = cv2.contourArea(contour)

        # Skip contours with small area
        if contour_area < min_contour_area:
            continue

        # Calculate the ink density for the contour
        ink_density = calculate_ink_density(image, contour)

        # Flag contours with ink density above the threshold as overwritten text
        if ink_density > threshold.any():
            # Draw a red rectangle around the flagged contour
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(flagged_image, (x, y), (x + w, y + h), (0, 0, 255), 2)

    return flagged_image

# Example usage:
# Load the image
image = cv2.imread('path_to_your_image.jpg')

# Flag the overwritten text based on ink density
flagged_image = flag_overwritten_text(image, threshold=0.5, min_contour_area=100)

# Display the flagged image
cv2.imshow('Flagged Overwritten Text', flagged_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
